## Code to have 2D Maxwell (with B in the direction z) using SummationByParts operators.

This is to be ported to the PIC code.

In [91]:
using Plots
using SummationByPartsOperators
using LinearAlgebra
using ArraysOfArrays

In [107]:
include("../../PIC/PIC-1D/aux_functions/aux_functions_grid.jl")
include("../../PIC/PIC-1D/aux_functions/aux_functions_E-B.jl")

get_Fields! (generic function with 1 method)

In [96]:
J = (10,20)
Box = (0.0,2.0,0.0,1.0)
dx = differentials(Box,J)
D = 2

xv = [(i-1)*dx[1] for i in 1:J[1]]
yv = [(i-1)*dx[2] for i in 1:J[2]];

In [97]:
Dx = periodic_derivative_operator(derivative_order=1, accuracy_order=6, xmin=Box[1], xmax=Box[2], N=J[1])
Dy = periodic_derivative_operator(derivative_order=1, accuracy_order=6, xmin=Box[3], xmax=Box[4], N=J[2])

Periodic first-derivative operator of order 6 on a grid in [0.0, 1.0] using 20 nodes, 
stencils with 3 nodes to the left, 3 nodes to the right, and coefficients of Fornberg (1998) 
  Calculation of Weights in Finite Difference Formulas. 
  SIAM Rev. 40.3, pp. 685-691.

### Initial Data

In [104]:
k = 2π*[3,3]
fe(x,y) = sin(k[1]*x)*sin(k[2]*y)
Ea = [(-1)^l*fe(xv[i],yv[j]) for l in 1:2, i in 1:J[1], j in 1:J[2]];
B = [fe(xv[i],yv[j]) for i in 1:J[1], j in 1:J[2]];


We upload the initial data onto the evolution vector.

In [137]:
u = Vector{Float64}(undef,3*J[1]*J[2])
du = Vector{Float64}(undef,3*J[1]*J[2])

F = reshape(u,3,J[1],J[2])

F[1:2,:,:] = Ea
F[3,:,:] = B

10×20 Matrix{Float64}:
  0.0   0.0           0.0          …  -0.0          -0.0
 -0.0  -0.475528     -0.559017         0.559017      0.475528
  0.0   0.769421      0.904508        -0.904508     -0.769421
 -0.0  -0.769421     -0.904508         0.904508      0.769421
  0.0   0.475528      0.559017        -0.559017     -0.475528
 -0.0  -5.94456e-16  -6.98825e-16  …   6.98825e-16   5.94456e-16
 -0.0  -0.475528     -0.559017         0.559017      0.475528
  0.0   0.769421      0.904508        -0.904508     -0.769421
 -0.0  -0.769421     -0.904508         0.904508      0.769421
  0.0   0.475528      0.559017        -0.559017     -0.475528

Check that it is OK for this toy data.

In [118]:
@show F[1,3,2]
@show F[2,3,2]
@show F[3,3,2]

F[1, 3, 2] = -0.7694208842938134
F[2, 3, 2] = 0.7694208842938134
F[3, 3, 2] = 0.7694208842938134


0.7694208842938134

Define the vectors to put the space derivatives.

In [138]:
function F!(du, u, t, par)
    u_a = reshape(u,(3,J...))
    Du_a = reshape(du,(3,J...))
    for i in 1:J[1]
        mul!(Du_a[1,i,:],Dy, u_a[3,i,:],one(eltype(u_a)))
        mul!(Du_a[3,i,:],Dy, u_a[1,i,:],one(eltype(u_a)))
    end
    for j in 1:J[2]
        mul!(Du_a[2,:,j],Dx, u_a[3,:,j],one(eltype(u_a)))
        mul!(Du_a[3,:,j],Dx, u_a[2,:,j],-one(eltype(u_a)),one(eltype(u_a)))
    end
    return du[:].=Du_a
end

F! (generic function with 1 method)

In [139]:
#du = similar(u)
du .= 0.0
t = 0.0
par = (0,)
F!(du,u,t,par)
du
#u

600-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0